In [1]:
import requests

In [68]:
requests.get("http://127.0.0.1:2443")

<Response [200]>

In [93]:
requests.get("http://127.0.0.1:2443", verify=False)

<Response [200]>

In [94]:
requests.get("http://127.0.0.1:28080").text

'Hello world'